In [13]:
import tensorflow as tf
keras = tf.keras
from keras import Sequential, Input
from keras.layers import Conv2D, Dense, AveragePooling2D, Flatten, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.optimizers import Adam
from keras.utils import plot_model
import os
from keras.applications import VGG16, ResNet50, MobileNetV2, EfficientNetB0, InceptionV3

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
base_dir = './catsdogs'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_dir,  
    target_size=(224, 224),  
    batch_size=20,
    class_mode='categorical',
)

Found 1598 images belonging to 2 classes.


In [5]:
val_generator = val_datagen.flow_from_directory(
    validation_dir,  
    target_size=(224, 224),  
    batch_size=20,
    class_mode='categorical',
)

Found 598 images belonging to 2 classes.


In [6]:
test_generator = test_datagen.flow_from_directory(
    test_dir,  
    target_size=(224, 224),  
    batch_size=20,
    class_mode='categorical',
)

Found 804 images belonging to 2 classes.


## VGG-16

In [7]:
vgg_model = VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
vgg_model.trainable = False
vgg_model.summary()

2023-08-27 16:22:50.022211: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:22:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-27 16:22:51.313054: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:22:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-27 16:22:51.313140: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:22:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-27 16:22:51.354507: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:22:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-27 16:22:51.354603: I tensorflow/compile

58889256/58889256 [==============================] - 5s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [8]:
custom_vgg_model = Sequential([
    vgg_model,
    Flatten(),
    Dense(2, activation='softmax')
])

custom_vgg_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 2)                 50178     
                                                                 
Total params: 14764866 (56.32 MB)
Trainable params: 50178 (196.01 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [9]:
custom_vgg_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate = 0.0001), metrics=['acc'])

In [10]:
vgg_history = custom_vgg_model.fit(train_generator, epochs=20, validation_data=val_generator)

Epoch 1/20


2023-08-27 16:23:04.581955: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-27 16:23:05.983571: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-27 16:23:11.367300: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x8e4ed90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-27 16:23:11.367377: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1060 6GB, Compute Capability 6.1
2023-08-27 16:23:12.025328: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-27 16:23:13.220516: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-27 16:23:13.414223: I ./tensorflow/compiler/jit/device_compi

80/80 [==============================] - 48s 475ms/step - loss: 0.4728 - acc: 0.7847 - val_loss: 0.3339 - val_acc: 0.8863
Epoch 2/20
80/80 [==============================] - 24s 306ms/step - loss: 0.2843 - acc: 0.9111 - val_loss: 0.2809 - val_acc: 0.9047
Epoch 3/20
80/80 [==============================] - 24s 293ms/step - loss: 0.2240 - acc: 0.9343 - val_loss: 0.2460 - val_acc: 0.9064
Epoch 4/20
80/80 [==============================] - 24s 298ms/step - loss: 0.1879 - acc: 0.9449 - val_loss: 0.2322 - val_acc: 0.9097
Epoch 5/20
80/80 [==============================] - 24s 299ms/step - loss: 0.1615 - acc: 0.9581 - val_loss: 0.2254 - val_acc: 0.9114
Epoch 6/20
80/80 [==============================] - 24s 300ms/step - loss: 0.1401 - acc: 0.9650 - val_loss: 0.2182 - val_acc: 0.9164
Epoch 7/20
80/80 [==============================] - 25s 311ms/step - loss: 0.1236 - acc: 0.9731 - val_loss: 0.2211 - val_acc: 0.9080
Epoch 8/20
80/80 [==============================] - 24s 298ms/step - loss: 0.111

In [11]:
custom_vgg_model.evaluate(test_generator)

41/41 [==============================] - 14s 350ms/step - loss: 0.2222 - acc: 0.9092


[0.22215589880943298, 0.9092040061950684]

## GoogleNet

In [14]:
google_model = InceptionV3(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
google_model.trainable = False
google_model.summary()

87910968/87910968 [==============================] - 6s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activatio

In [15]:
custom_google_model = Sequential([
    google_model,
    Flatten(),
    Dense(2, activation='softmax')
])

custom_google_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten_1 (Flatten)         (None, 51200)             0         
                                                                 
 dense_1 (Dense)             (None, 2)                 102402    
                                                                 
Total params: 21905186 (83.56 MB)
Trainable params: 102402 (400.01 KB)
Non-trainable params: 21802784 (83.17 MB)
_________________________________________________________________


In [16]:
custom_google_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate = 0.0001), metrics=['acc'])

In [17]:
google_history = custom_google_model.fit(train_generator, epochs=20, validation_data=val_generator)

Epoch 1/20
80/80 [==============================] - 35s 345ms/step - loss: 0.1401 - acc: 0.9499 - val_loss: 0.0430 - val_acc: 0.9900
Epoch 2/20
80/80 [==============================] - 25s 309ms/step - loss: 0.0174 - acc: 0.9937 - val_loss: 0.0885 - val_acc: 0.9732
Epoch 3/20
80/80 [==============================] - 25s 307ms/step - loss: 0.0060 - acc: 0.9981 - val_loss: 0.0732 - val_acc: 0.9783
Epoch 4/20
80/80 [==============================] - 25s 310ms/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.0556 - val_acc: 0.9866
Epoch 5/20
80/80 [==============================] - 25s 318ms/step - loss: 7.1671e-04 - acc: 1.0000 - val_loss: 0.0528 - val_acc: 0.9866
Epoch 6/20
80/80 [==============================] - 25s 308ms/step - loss: 5.3999e-04 - acc: 1.0000 - val_loss: 0.0519 - val_acc: 0.9866
Epoch 7/20
80/80 [==============================] - 26s 320ms/step - loss: 4.5584e-04 - acc: 1.0000 - val_loss: 0.0533 - val_acc: 0.9849
Epoch 8/20
80/80 [==============================] - 29s 3

In [18]:
custom_google_model.evaluate(test_generator)

41/41 [==============================] - 12s 286ms/step - loss: 0.0510 - acc: 0.9876


[0.05097343772649765, 0.9875621795654297]

## ResNet50

In [19]:
res_model = ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
res_model.trainable = False
res_model.summary()

94765736/94765736 [==============================] - 7s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_3[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                      

In [20]:
custom_res_model = Sequential([
    res_model,
    Flatten(),
    Dense(2, activation='softmax')
])

custom_res_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_2 (Flatten)         (None, 100352)            0         
                                                                 
 dense_2 (Dense)             (None, 2)                 200706    
                                                                 
Total params: 23788418 (90.75 MB)
Trainable params: 200706 (784.01 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [21]:
custom_res_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate = 0.0001), metrics=['acc'])

In [22]:
res_history = custom_res_model.fit(train_generator, epochs=20, validation_data=val_generator)

Epoch 1/20
80/80 [==============================] - 31s 326ms/step - loss: 0.7048 - acc: 0.5788 - val_loss: 0.8036 - val_acc: 0.5418
Epoch 2/20
80/80 [==============================] - 24s 300ms/step - loss: 0.6595 - acc: 0.6026 - val_loss: 0.6229 - val_acc: 0.6639
Epoch 3/20
80/80 [==============================] - 25s 315ms/step - loss: 0.6013 - acc: 0.6721 - val_loss: 0.6424 - val_acc: 0.6221
Epoch 4/20
80/80 [==============================] - 28s 343ms/step - loss: 0.6102 - acc: 0.6539 - val_loss: 0.6558 - val_acc: 0.6120
Epoch 5/20
80/80 [==============================] - 28s 351ms/step - loss: 0.6256 - acc: 0.6527 - val_loss: 0.6098 - val_acc: 0.6806
Epoch 6/20
80/80 [==============================] - 26s 319ms/step - loss: 0.5896 - acc: 0.6796 - val_loss: 0.7400 - val_acc: 0.5602
Epoch 7/20
80/80 [==============================] - 27s 337ms/step - loss: 0.5802 - acc: 0.6909 - val_loss: 0.6142 - val_acc: 0.6722
Epoch 8/20
80/80 [==============================] - 26s 325ms/step - 

In [23]:
custom_res_model.evaluate(test_generator)

41/41 [==============================] - 11s 261ms/step - loss: 0.5743 - acc: 0.6928


[0.5742542147636414, 0.6927860975265503]

## EfficientNetB0

In [24]:
eff_model = EfficientNetB0(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
eff_model.trainable = False
eff_model.summary()

16705208/16705208 [==============================] - 3s 0us/step
Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_4[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescal

In [25]:
custom_eff_model = Sequential([
    eff_model,
    Flatten(),
    Dense(2, activation='softmax')
])

custom_eff_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, 7, 7, 1280)        4049571   
 )                                                               
                                                                 
 flatten_3 (Flatten)         (None, 62720)             0         
                                                                 
 dense_3 (Dense)             (None, 2)                 125442    
                                                                 
Total params: 4175013 (15.93 MB)
Trainable params: 125442 (490.01 KB)
Non-trainable params: 4049571 (15.45 MB)
_________________________________________________________________


In [26]:
custom_eff_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate = 0.0001), metrics=['acc'])

In [27]:
eff_history = custom_eff_model.fit(train_generator, epochs=20, validation_data=val_generator)

Epoch 1/20


2023-08-27 18:29:50.991152: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_3/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


80/80 [==============================] - 40s 397ms/step - loss: 0.7570 - acc: 0.5031 - val_loss: 0.6967 - val_acc: 0.5000
Epoch 2/20
80/80 [==============================] - 27s 336ms/step - loss: 0.7395 - acc: 0.5307 - val_loss: 0.7522 - val_acc: 0.5000
Epoch 3/20
80/80 [==============================] - 27s 332ms/step - loss: 0.7366 - acc: 0.5006 - val_loss: 0.8219 - val_acc: 0.5000
Epoch 4/20
80/80 [==============================] - 25s 311ms/step - loss: 0.7390 - acc: 0.4931 - val_loss: 0.6914 - val_acc: 0.5669
Epoch 5/20
80/80 [==============================] - 25s 308ms/step - loss: 0.7258 - acc: 0.5106 - val_loss: 0.7088 - val_acc: 0.5000
Epoch 6/20
80/80 [==============================] - 25s 316ms/step - loss: 0.7271 - acc: 0.5169 - val_loss: 0.7008 - val_acc: 0.5000
Epoch 7/20
80/80 [==============================] - 25s 315ms/step - loss: 0.7312 - acc: 0.5081 - val_loss: 0.7825 - val_acc: 0.5000
Epoch 8/20
80/80 [==============================] - 27s 337ms/step - loss: 0.785

In [28]:
custom_eff_model.evaluate(test_generator)

41/41 [==============================] - 11s 260ms/step - loss: 0.6945 - acc: 0.5000


[0.6944627165794373, 0.5]

## MobileNetV2


In [29]:
mobile_model = MobileNetV2(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
mobile_model.trainable = False
mobile_model.summary()

9406464/9406464 [==============================] - 1s 0us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_5[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReL

In [30]:
custom_mobile_model = Sequential([
    mobile_model,
    Flatten(),
    Dense(2, activation='softmax')
])

custom_mobile_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 flatten_4 (Flatten)         (None, 62720)             0         
                                                                 
 dense_4 (Dense)             (None, 2)                 125442    
                                                                 
Total params: 2383426 (9.09 MB)
Trainable params: 125442 (490.01 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [31]:
custom_mobile_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate = 0.0001), metrics=['acc'])

In [32]:
mobile_history = custom_mobile_model.fit(train_generator, epochs=20, validation_data=val_generator)

Epoch 1/20
80/80 [==============================] - 29s 327ms/step - loss: 0.1696 - acc: 0.9399 - val_loss: 0.0990 - val_acc: 0.9682
Epoch 2/20
80/80 [==============================] - 24s 305ms/step - loss: 0.0127 - acc: 0.9950 - val_loss: 0.0725 - val_acc: 0.9783
Epoch 3/20
80/80 [==============================] - 25s 307ms/step - loss: 0.0026 - acc: 1.0000 - val_loss: 0.0555 - val_acc: 0.9849
Epoch 4/20
80/80 [==============================] - 25s 312ms/step - loss: 6.7159e-04 - acc: 1.0000 - val_loss: 0.0549 - val_acc: 0.9833
Epoch 5/20
80/80 [==============================] - 25s 319ms/step - loss: 4.9279e-04 - acc: 1.0000 - val_loss: 0.0551 - val_acc: 0.9833
Epoch 6/20
80/80 [==============================] - 27s 338ms/step - loss: 4.0310e-04 - acc: 1.0000 - val_loss: 0.0545 - val_acc: 0.9833
Epoch 7/20
80/80 [==============================] - 25s 314ms/step - loss: 3.4763e-04 - acc: 1.0000 - val_loss: 0.0553 - val_acc: 0.9833
Epoch 8/20
80/80 [==============================] - 2

In [33]:
custom_mobile_model.evaluate(test_generator)

41/41 [==============================] - 12s 285ms/step - loss: 0.0871 - acc: 0.9789


[0.08708406239748001, 0.9788557291030884]